<a href="https://www.kaggle.com/code/adityabayhaqie/deepkpg-adit?scriptVersionId=221621417" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [5]:
!rm -r /kaggle/working/*
!git clone https://github.com/uclanlp/DeepKPG

rm: cannot remove '/kaggle/working/*': No such file or directory
Cloning into 'DeepKPG'...
fatal: unable to access 'https://github.com/uclanlp/DeepKPG/': Could not resolve host: github.com


In [2]:
!pip install -q deepspeed
!pip install -q datasets
!pip install -q evaluate
!pip install -q gdown

ERROR: Could not find a version that satisfies the requirement deepspeed (from versions: none)
ERROR: No matching distribution found for deepspeed
^C


In [3]:
!gdown https://drive.google.com/drive/folders/1l91-ptaTNPq0jIY6Pahi370_52w0IlGM?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1pxgCrkisCZ6M0ozwfiXPOV4gquqk2E4z
From (redirected): https://drive.google.com/uc?id=1pxgCrkisCZ6M0ozwfiXPOV4gquqk2E4z&confirm=t&uuid=fdad1093-fc4e-47b0-9323-6c4b09eecbac
To: /kaggle/working/fullDataset.zip
100%|███████████████████████████████████████| 1.50G/1.50G [00:07<00:00, 212MB/s]


In [4]:
%cd /kaggle/working

/kaggle/working


In [5]:
!unzip /kaggle/working/fullDataset.zip

Archive:  /kaggle/working/fullDataset.zip
   creating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/
   creating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/processed/
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/processed/train.json  
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/processed/valid.json  
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/processed/test.json  
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/processed/vocab.txt  
   creating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/fairseq/
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/fairseq/train.source  
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/fairseq/train.target  
  inflating: content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original/fairseq/valid.source  
  inflating: conte

In [6]:
!mkdir -p /kaggle/working/DeepKPG/data 

!mv /kaggle/working/content/drive/MyDrive/datasetFinal/kp20k_full_dataset_original /kaggle/working/DeepKPG/data/kp20k-original
# !cp -r /kaggle/input/dataset/HASBI_kp20k-text-bertopic-10k /kaggle/working/DeepKPG/data/kp20k-text-bertopic
!mv /kaggle/working/content/drive/MyDrive/datasetFinal/kp20k_full_dataset_title_topic /kaggle/working/DeepKPG/data/kp20k-title-bertopic
!cp -r /kaggle/input/test-data/* /kaggle/working/DeepKPG/data

In [7]:
%cd /kaggle/working/DeepKPG/sequence_generation/seq2seq

/kaggle/working/DeepKPG/sequence_generation/seq2seq


In [8]:
%%writefile /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_finetune_kpgen_seq2seq_hf.py

#!/usr/bin/env python
# coding=utf-8
# based on https://github.com/huggingface/transformers/blob/v4.16.2/examples/pytorch/summarization/run_summarization.py
"""
Fine-tuning a 🤗 Transformers model on keyphrase generation.
"""

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import nltk
import numpy as np
import random
from datasets import load_dataset
from evaluate import load as load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from transformers.integrations import TensorBoardCallback


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.15.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/summarization/requirements.txt")

logger = logging.getLogger(__name__)
consoleHandler = logging.StreamHandler()
logger.addHandler(consoleHandler)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    resize_position_embeddings: Optional[bool] = field(
        default=None,
        metadata={
            "help": "Whether to automatically resize the position embeddings if `max_source_length` exceeds "
            "the model's position embeddings."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    src_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the source texts (for keyphrase generation)."},
    )
    tgt_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the keyphrases (for keyphrase generation)."},
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines or csv file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (rouge) on "
            "(a jsonlines or csv file)."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on " "(a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": "The maximum total sequence length for target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total sequence length for validation target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
            "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
            "during ``evaluate`` and ``predict``."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    num_beams: Optional[int] = field(
        default=1,
        metadata={
            "help": "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
            "which is used during ``evaluate`` and ``predict``."
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default=None, metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length


kpgen_name_mapping = {
    "kp20k": ("src", "tgt"),
}


def load_and_preprocess_datasets(model_args, data_args, training_args, tokenizer, model):
    # Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files this script will use the first column for the full texts and the second column for the
    # keyphrases (unless you specify column names for this with the `src_column` and `tgt_column` arguments).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    
    prefix = data_args.source_prefix if data_args.source_prefix is not None else ""

    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir
        )
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
            extension = data_args.train_file.split(".")[-1]
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
            extension = data_args.validation_file.split(".")[-1]
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
            extension = data_args.test_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)

    # Preprocessing the datasets.
    # We need to tokenize inputs and targets.
    if training_args.do_train:
        column_names = raw_datasets["train"].column_names
    elif training_args.do_eval:
        column_names = raw_datasets["validation"].column_names
    elif training_args.do_predict:
        column_names = raw_datasets["test"].column_names
    else:
        logger.info("There is nothing to do. Please pass `do_train`, `do_eval` and/or `do_predict`.")
        return

    # Get the column names for input/target.
    dataset_columns = kpgen_name_mapping.get(data_args.dataset_name, None)
    if data_args.src_column is None:
        src_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
    else:
        src_column = data_args.src_column
        if src_column not in column_names:
            raise ValueError(
                f"--src_column' value '{data_args.src_column}' needs to be one of: {', '.join(column_names)}"
            )
    if data_args.tgt_column is None:
        tgt_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
    else:
        tgt_column = data_args.tgt_column
        if tgt_column not in column_names:
            raise ValueError(
                f"--tgt_column' value '{data_args.tgt_column}' needs to be one of: {', '.join(column_names)}"
            )

    # Temporarily set max_target_length for training.
    max_target_length = data_args.max_target_length
    padding = "max_length" if data_args.pad_to_max_length else False

    if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
        logger.warning(
            "label_smoothing is enabled but the `prepare_decoder_input_ids_from_labels` method is not defined for"
            f"`{model.__class__.__name__}`. This will lead to loss being calculated twice and will take up more memory"
        )

    def preprocess_function(examples):
        # remove pairs where at least one record is None
        inputs, targets = [], []
        for i in range(len(examples[src_column])):
            if examples[src_column][i] is not None and examples[tgt_column][i] is not None:
                inputs.append(examples[src_column][i])
                targets.append(examples[tgt_column][i])

        inputs = [prefix + inp for inp in inputs]
        model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    train_dataset, eval_dataset, predict_dataset = None, None, None
    if training_args.do_train:
        if "train" not in raw_datasets:
            raise ValueError("--do_train requires a train dataset")
        train_dataset = raw_datasets["train"]
        if data_args.max_train_samples is not None:
            # train_dataset = train_dataset.select(range(data_args.max_train_samples))
            rand_data_idx = random.sample(list(range(len(train_dataset))), data_args.max_train_samples)
            train_dataset = train_dataset.select(rand_data_idx)
        with training_args.main_process_first(desc="train dataset map pre-processing"):
            train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

    if training_args.do_eval:
        max_target_length = data_args.val_max_target_length
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = raw_datasets["validation"]
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
        with training_args.main_process_first(desc="validation dataset map pre-processing"):
            eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on validation dataset",
            )

    if training_args.do_predict:
        max_target_length = data_args.val_max_target_length
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = raw_datasets["test"]
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))
        with training_args.main_process_first(desc="prediction dataset map pre-processing"):
            predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )

    return train_dataset, eval_dataset, predict_dataset


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    # log to both file and stderr
    fileHandler = logging.FileHandler("{}/train.log".format(training_args.output_dir))
    logger.addHandler(fileHandler)
    log_level = training_args.get_process_log_level()
    logger.setLevel(log_level)
    datasets.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")

    if data_args.source_prefix is None and model_args.model_name_or_path in [
        "t5-small",
        "t5-base",
        "t5-large",
        "t5-3b",
        "t5-11b",
    ]:
        logger.warning(
            "You're running a t5 model but didn't provide a source prefix, which is the expected, e.g. with "
            "`--source_prefix 'summarize: ' `"
        )

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    if 't5' not in model_args.model_name_or_path:
        model.resize_token_embeddings(len(tokenizer))

    if model.config.decoder_start_token_id is None:
        raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

    if (
        hasattr(model.config, "max_position_embeddings")
        and model.config.max_position_embeddings < data_args.max_source_length
    ):
        if model_args.resize_position_embeddings is None:
            logger.warning(
                f"Increasing the model's number of position embedding vectors from {model.config.max_position_embeddings} "
                f"to {data_args.max_source_length}."
            )
            model.resize_position_embeddings(data_args.max_source_length)
        elif model_args.resize_position_embeddings:
            model.resize_position_embeddings(data_args.max_source_length)
        else:
            raise ValueError(
                f"`--max_source_length` is set to {data_args.max_source_length}, but the model only has {model.config.max_position_embeddings}"
                f" position encodings. Consider either reducing `--max_source_length` to {model.config.max_position_embeddings} or to automatically "
                "resize the model's position encodings by passing `--resize_position_embeddings`."
            )

    # load and preprocess the datasets
    train_dataset, eval_dataset, predict_dataset = load_and_preprocess_datasets(model_args, data_args, training_args, tokenizer, model)

    # Data collator
    label_pad_token_id = tokenizer.pad_token_id
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if training_args.fp16 else None,
    )

    # Metric
    # metric = load_metric("rouge")
    def postprocess_text_kpgen(preds, labels):
        preds = [[x.strip() for x in pred.strip().split(';')] for pred in preds]
        labels = [[x.strip() for x in label.strip().split(';')] for label in labels]
        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        # preds= np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        if data_args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        decoded_preds, decoded_labels = postprocess_text_kpgen(decoded_preds, decoded_labels)

        # batch-wise f1
        kp2scores = {}
        for cur_label in set([l for case in decoded_labels for l in case]):
            kp2scores[cur_label] = {"tp": 0, "fp": 0, "fn": 0, "label_count": 0, "pred_count": 0, "precision": 0, "recall": 0, "f1": 0}
        for cur_labels, cur_preds in zip(decoded_labels, decoded_preds):
            cur_preds = set(cur_preds)
            cur_labels = set(cur_labels)
            for cur_label in cur_labels:
                kp2scores[cur_label]["label_count"] += 1
                if cur_label in cur_preds:
                    kp2scores[cur_label]["tp"] += 1
                else:
                    kp2scores[cur_label]["fn"] += 1
            for cur_pred in cur_preds:
                if cur_pred in kp2scores:
                    kp2scores[cur_pred]["pred_count"] += 1
                    if cur_pred not in cur_labels:
                        kp2scores[cur_pred]["fp"] += 1
        for cur_label in kp2scores.keys():
            kp2scores[cur_label]['precision'] = (kp2scores[cur_label]["tp"] / kp2scores[cur_label]["pred_count"]) if kp2scores[cur_label]["pred_count"] != 0 else 0
            kp2scores[cur_label]['recall'] = (kp2scores[cur_label]["tp"] / kp2scores[cur_label]["label_count"]) if kp2scores[cur_label]["pred_count"] != 0 else 0 
            kp2scores[cur_label]['f1'] = ((kp2scores[cur_label]['precision'] + kp2scores[cur_label]['recall']) / 2 * kp2scores[cur_label]['recall'] * kp2scores[cur_label]['precision']) if kp2scores[cur_label]['recall'] * kp2scores[cur_label]['precision'] != 0 else 0

        result = {'f1': np.mean([x['f1'] for x in kp2scores.values()])}
        return result

    # Initialize our Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.predict_with_generate else None,
        callbacks=[TensorBoardCallback()],
    )

    # Training
    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # Saves the tokenizer too for easy upload

        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    results = {}
    max_length = (
        training_args.generation_max_length
        if training_args.generation_max_length is not None
        else data_args.val_max_target_length
    )
    num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
    if training_args.do_eval:
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    """
    if training_args.do_predict:
        logger.info("*** Predict ***")

        predict_results = trainer.predict(
            predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=num_beams
        )
        metrics = predict_results.metrics
        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        if trainer.is_world_process_zero():
            if training_args.predict_with_generate:
                predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                predictions = [pred.strip() for pred in predictions]
                output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
                with open(output_prediction_file, "w") as writer:
                    writer.write("\n".join(predictions))

    kwargs = {"finetuned_from": model_args.model_name_or_path, "tasks": "summarization"}
    if data_args.dataset_name is not None:
        kwargs["dataset_tags"] = data_args.dataset_name
        if data_args.dataset_config_name is not None:
            kwargs["dataset_args"] = data_args.dataset_config_name
            kwargs["dataset"] = f"{data_args.dataset_name} {data_args.dataset_config_name}"
        else:
            kwargs["dataset"] = data_args.dataset_name

    if training_args.push_to_hub:
        trainer.push_to_hub(**kwargs)
    else:
        trainer.create_model_card(**kwargs)
    """

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()

Overwriting /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_finetune_kpgen_seq2seq_hf.py


In [9]:
%%writefile /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_decode_kpgen_seq2seq_hf.py
#!/usr/bin/env python
# coding=utf-8
# based on https://github.com/huggingface/transformers/blob/v4.16.2/examples/pytorch/summarization/run_summarization_no_trainer.py
"""
Decoding a 🤗 Transformers model on keyphrase generation.
"""

import argparse
import logging
import math
import json
import os
import sys
import random
from pathlib import Path

import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset
from evaluate import load as load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from huggingface_hub import Repository
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    SchedulerType,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import get_full_repo_name, is_offline_mode
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)
consoleHandler = logging.StreamHandler()
logger.addHandler(consoleHandler)

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/summarization/requirements.txt")

# You should update this to your particular problem to have better documentation of `model_type`
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)


kpgen_name_mapping = {
    "kp20k": ("src", "tgt"),
}


def postprocess_text_kpgen(preds, labels, predsonly=False):
    preds = [[x.strip() for x in pred.strip().split(';')] for pred in preds]
    if not predsonly:
        labels = [[x.strip() for x in label.strip().split(';')] for label in labels]
    return preds, labels


def parse_args():
    parser = argparse.ArgumentParser(description="Finetune a transformers model on keyphrase generation")
    parser.add_argument(
        "--dataset_name",
        type=str,
        default=None,
        help="The name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--dataset_config_name",
        type=str,
        default=None,
        help="The configuration name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--test_file", type=str, default=None, help="A csv or a json file containing the test data."
    )
    parser.add_argument(
        "--ignore_pad_token_for_loss",
        type=bool,
        default=True,
        help="Whether to ignore the tokens corresponding to " "padded labels in the loss computation or not.",
    )
    parser.add_argument(
        "--max_source_length",
        type=int,
        default=1024,
        help="The maximum total input sequence length after "
        "tokenization.Sequences longer than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument(
        "--source_prefix",
        type=str,
        default=None,
        help="A prefix to add before every source text " "(useful for T5 models).",
    )
    parser.add_argument(
        "--preprocessing_num_workers",
        type=int,
        default=None,
        help="The number of processes to use for the preprocessing.",
    )
    parser.add_argument(
        "--overwrite_cache", type=bool, default=None, help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument(
        "--max_target_length",
        type=int,
        default=128,
        help="The maximum total sequence length for target text after "
        "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
        "during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--val_max_target_length",
        type=int,
        default=None,
        help="The maximum total sequence length for validation "
        "target text after tokenization.Sequences longer than this will be truncated, sequences shorter will be "
        "padded. Will default to `max_target_length`.This argument is also used to override the ``max_length`` "
        "param of ``model.generate``, which is used during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--max_length",
        type=int,
        default=128,
        help=(
            "The maximum total input sequence length after tokenization. Sequences longer than this will be truncated,"
            " sequences shorter will be padded if `--pad_to_max_lengh` is passed."
        ),
    )
    parser.add_argument(
        "--num_beams",
        type=int,
        default=1,
        help="Number of beams to use for evaluation. This argument will be "
        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--pad_to_max_length",
        action="store_true",
        help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",
    )
    parser.add_argument(
        "--model_name_or_path",
        type=str,
        help="Path to pretrained model or model identifier from huggingface.co/models.",
        required=True,
    )
    # parser.add_argument(
    #     "--config_name",
    #     type=str,
    #     default=None,
    #     help="Pretrained config name or path if not the same as model_name",
    # )
    parser.add_argument(
        "--tokenizer_name",
        type=str,
        default=None,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--src_column",
        type=str,
        default=None,
        help="The name of the column in the datasets containing the source texts (for keyphrase generation).",
    )
    parser.add_argument(
        "--tgt_column",
        type=str,
        default=None,
        help="The name of the column in the datasets containing the keyphrases (for keyphrase generation).",
    )
    parser.add_argument(
        "--use_slow_tokenizer",
        action="store_true",
        help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
    )
    parser.add_argument(
        "--per_device_eval_batch_size",
        type=int,
        default=8,
        help="Batch size (per device) for the evaluation dataloader.",
    )
    parser.add_argument("--output_dir", type=str, required=True, help="Dir for saving eval log.")
    parser.add_argument("--output_file_name", type=str, required=True, help="Output hypothesis file name.")
    parser.add_argument("--write_raw", type=bool, default=True, help="Write raw text lines (True) or json with phrases splitted (False).")

    args = parser.parse_args()

    # Sanity checks
    if args.dataset_name is None and args.test_file is None:
        raise ValueError("Need either a dataset name or a testing file.")
    else:
        if args.test_file is not None:
            extension = args.test_file.split(".")[-1]
            assert extension in ["csv", "json"], "`test_file` should be a csv or a json file."

    return args


def get_test_dataset(args, accelerator, tokenizer):
    if args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
    else:
        assert args.test_file is not None
        data_files = {}
        data_files["test"] = args.test_file
        extension = args.test_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files)

    prefix = args.source_prefix if args.source_prefix is not None else ""

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    column_names = raw_datasets["test"].column_names

    # Get the column names for input/target.
    dataset_columns = kpgen_name_mapping.get(args.dataset_name, None)
    if args.src_column is None:
        src_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
    else:
        src_column = args.src_column
        if src_column not in column_names:
            raise ValueError(
                f"--src_column' value '{args.src_column}' needs to be one of: {', '.join(column_names)}"
            )
    if args.tgt_column is None:
        tgt_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
    else:
        tgt_column = args.tgt_column
        if tgt_column not in column_names:
            raise ValueError(
                f"--tgt_column' value '{args.tgt_column}' needs to be one of: {', '.join(column_names)}"
            )

    # Temporarily set max_target_length for training.
    max_target_length = args.max_target_length
    padding = "max_length" if args.pad_to_max_length else False

    def preprocess_function(examples):
        inputs = examples[src_column]
        targets = examples[tgt_column]        
        
        inputs = [prefix + inp for inp in inputs]
        model_inputs = tokenizer(inputs, max_length=args.max_source_length, padding=padding, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    with accelerator.main_process_first():
        processed_datasets = raw_datasets.map(
            preprocess_function,
            batched=True,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )

    return processed_datasets["test"]


def main():
    args = parse_args()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    logger.info("Inference on device: " + str(device))

    # log to both file and stderr
    fileHandler = logging.FileHandler("{}/test.log".format(args.output_dir))
    logger.addHandler(fileHandler)

    if args.source_prefix is None and args.model_name_or_path in [
        "t5-small",
        "t5-base",
        "t5-large",
        "t5-3b",
        "t5-11b",
    ]:
        logger.warning(
            "You're running a t5 model but didn't provide a source prefix, which is the expected, e.g. with "
            "`--source_prefix 'summarize: ' `"
        )
    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # Handle the repository creation
    if accelerator.is_main_process:
        if args.output_dir is not None:
            os.makedirs(args.output_dir, exist_ok=True)
    accelerator.wait_for_everyone()

    config = AutoConfig.from_pretrained(args.model_name_or_path)
    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
    else:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
    model = AutoModelForSeq2SeqLM.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
        )
    model = model.to(device)
    logger.info("Loaded config, tokenizer, and model from " + args.model_name_or_path)
    logger.info("Tokenizer: " + str(type(tokenizer)))

    model.resize_token_embeddings(len(tokenizer))
    if model.config.decoder_start_token_id is None:
        raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

    # get datasets
    test_dataset = get_test_dataset(args, accelerator, tokenizer)

    # Log a few random samples from the training set:
    for index in random.sample(range(len(test_dataset)), 1):
        logger.info(f"Sample {index} of the testing set: {test_dataset[index]}.")

    label_pad_token_id = tokenizer.pad_token_id
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if accelerator.use_fp16 else None,
    )

    eval_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)

    logger.info("***** Running decode *****")
    logger.info(f"  Num examples = {len(test_dataset)}")
    logger.info(f"  Total eval batch size = {args.per_device_eval_batch_size}")

    model.eval()
    if args.val_max_target_length is None:
        args.val_max_target_length = args.max_target_length

    gen_kwargs = {
        "max_length": args.val_max_target_length if args is not None else config.max_length,
        "num_beams": args.num_beams,
    }

    valid_preds = []
    for step, batch in tqdm(enumerate(eval_dataloader), desc='Decoding'):
        with torch.no_grad():
            batch = batch.to(device)

            # accumulate output for f1 score calculation
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                **gen_kwargs,
            )
            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]

            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            if not args.write_raw:
                decoded_preds, _ = postprocess_text_kpgen(decoded_preds, None, predsonly=True)

            valid_preds.extend(decoded_preds)

    accelerator.wait_for_everyone()
    with open(args.output_file_name, 'w') as f:
        for pred in valid_preds:
            if args.write_raw:
                f.write(pred)
            else:
                f.write(json.dumps(pred))
            f.write('\n')


if __name__ == "__main__":
    main()

Overwriting /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_decode_kpgen_seq2seq_hf.py


In [10]:
!wandb login ea1a63c41da8fb55aaa724903767c76bbed85c3d

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
%%writefile /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_bart_large.sh
#!/usr/bin/env bash

export PYTHONIOENCODING=utf-8;
HOME_DIR=`/kaggle/working`

# Confirm GPUs, model, and dataset before training
export CUDA_VISIBLE_DEVICES=0,1

MODEL_NAME_OR_PATH="facebook/bart-base"
MODEL_NAME_SHORT="BART"      # for creating the output dir

DATASET=kp20k-original


function train () {
    BATCH_SIZE_PER_GPU=12
    GRAD_ACCUMULATION_STEPS=4
    N_EPOCHS=1
    N_WARMUP_STEPS=100
    N_EVAL_STEPS=5529
    LR=6e-5
    LR_schedule='polynomial'    # 'linear'
    # LR_schedule='constant'
    SEED=1234
    
    OUT_DIR=/kaggle/working/DeepKPG/lm_kpgen_experiments/${MODEL_NAME_SHORT}/$(date +'%Y%m%d-%H%M')_${DATASET}_checkpoints_${MODEL_NAME_SHORT}_lr${LR}_${LR_schedule}_seed${SEED}_epoch${N_EPOCHS}
    mkdir -p ${OUT_DIR}/code_backup
    cp *.py *.sh ${OUT_DIR}/code_backup


    # give the argument --num_gpus=X to deepspeed if we don't use CUDA_VISIBLE_DEVICES   
    python sequence_generation/seq2seq/run_finetune_kpgen_seq2seq_hf.py \
        --output_dir $OUT_DIR \
        --model_name_or_path ${MODEL_NAME_OR_PATH} \
        --do_train \
        --do_eval \
        --train_file "/kaggle/working/DeepKPG/data/${DATASET}/json/train.json" \
        --validation_file "/kaggle/working/DeepKPG/data/${DATASET}/json/valid.json" \
        --src_column "src" \
        --tgt_column "tgt" \
        --per_device_train_batch_size ${BATCH_SIZE_PER_GPU} \
        --per_device_eval_batch_size ${BATCH_SIZE_PER_GPU} \
        --gradient_accumulation_steps ${GRAD_ACCUMULATION_STEPS} \
        --num_train_epochs ${N_EPOCHS} \
        --learning_rate ${LR} \
        --lr_scheduler_type ${LR_schedule} \
        --warmup_steps ${N_WARMUP_STEPS} \
        --logging_strategy 'steps' \
        --logging_steps 500 \
        --evaluation_strategy 'steps' \
        --eval_steps ${N_EVAL_STEPS} \
        --save_strategy 'steps' \
        --save_steps ${N_EVAL_STEPS} \
        --save_total_limit 1 \
        --load_best_model_at_end true \
        --overwrite_output_dir \
        --predict_with_generate \
        --seed ${SEED} \
        --fp16 true \
        --half_precision_backend "auto"

}


function decode () {
    EVAL_DATASET=$1
    BATCH_SIZE_PER_GPU=32

    python sequence_generation/seq2seq/run_decode_kpgen_seq2seq_hf.py \
        --model_name_or_path $OUT_DIR \
        --tokenizer_name $OUT_DIR \
        --test_file "data/${EVAL_DATASET}/json/test.json" \
        --src_column "src" \
        --tgt_column "tgt" \
        --num_beams 1 \
        --per_device_eval_batch_size $BATCH_SIZE_PER_GPU \
        --output_dir $OUT_DIR \
        --output_file_name "$OUT_DIR/${EVAL_DATASET}_hypotheses.txt" 

}


function evaluate () {
    EVAL_DATASET=$1

    python -W ignore utils/evaluate.py \
        --src_dir data/${EVAL_DATASET}/fairseq \
        --file_prefix $OUT_DIR/${EVAL_DATASET} \
        --tgt_dir $OUT_DIR \
        --log_file $EVAL_DATASET \
        --k_list 5 M;
}


# if [[ $DATASET == 'kp20k-title-bertopic-10k' ]] ; then # here
if [[ $DATASET == 'kp20k-original' ]] ; then # here
    DATA_DIR_PREFIX="/data/scikp"
    train
    for dataset in kp20k-original inspec krapivin nus semeval; do
        decode $dataset
        evaluate $dataset
    done	
elif [[ $DATASET =~ ^(kpbiomed-small|kpbiomed-medium|kpbiomed-large)$ ]]; then
    DATA_DIR_PREFIX="$/data/kpbiomed"
    train
    decode $DATASET
    evaluate $DATASET
else
    DATA_DIR_PREFIX="$/data"
    train
    decode $DATASET
    evaluate $DATASET
fi

Overwriting /kaggle/working/DeepKPG/sequence_generation/seq2seq/run_bart_large.sh


In [12]:
import os

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [13]:
# Pindah ke direktori yang berisi script
%cd /kaggle/working/DeepKPG

# Jalankan script
!bash sequence_generation/seq2seq/run_bart_large.sh

/kaggle/working/DeepKPG
sequence_generation/seq2seq/run_bart_large.sh: line 4: /kaggle/working: Is a directory
cp: cannot stat '*.py': No such file or directory
2025-01-29 15:22:18.930548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-29 15:22:19.145178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-29 15:22:19.199978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` in